In [1]:
import pandas as pd
import openai

# Integrate Service/Ward info

This section will integrate the service-file to match the structure of the note as much as possible. 

In [2]:
challenge_data_fp = "/gpfs/gibbs/project/rtaylor/shared/DischargeMe/public/"


In [3]:
from datasets import load_dataset

#train_dataset = load_dataset('json', data_files=challenge_data_fp + "train/simple_train.json", split='train')
#eval_dataset = load_dataset('json', data_files=challenge_data_fp + "valid/simple_valid.json", split='train')
test_dataset = load_dataset('json', data_files="/home/vs428/Documents/DischargeMe/hail-dischargeme/notebooks/data_processing/simple_test.json", split="train")

In [4]:
target_test = pd.read_csv(challenge_data_fp + "test_phase_1/discharge_target.csv.gz", keep_default_na=False)


In [5]:
# ward transfers
transfers = pd.read_pickle('/gpfs/gibbs/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_transfers.pkl')

# higher-level services (ICU, CARD, etc)
services = pd.read_pickle('/gpfs/gibbs/project/rtaylor/shared/DischargeMe/mimiciv/hosp/cohort_services.pkl')


In [6]:
transfers = transfers[transfers['eventtype'] != "discharge"]

In [7]:
transfers = transfers.sort_values(['hadm_id', "intime"])

In [8]:
discharging_transfer = transfers.groupby("hadm_id").last().reset_index()

In [9]:
discharging_transfer['careunit'].value_counts()

careunit
Medicine                                            59400
Med/Surg                                            20460
Emergency Department Observation                    20027
Medicine/Cardiology                                 15441
Neurology                                           14445
Hematology/Oncology                                 13244
Transplant                                          12433
Vascular                                            10642
Med/Surg/Trauma                                     10496
Med/Surg/GYN                                         9831
Hematology/Oncology Intermediate                     4937
Surgery/Trauma                                       4525
Cardiac Surgery                                      3442
Surgery                                              2980
Medicine/Cardiology Intermediate                     2626
Psychiatry                                           1823
Medical/Surgical (Gynecology)                        1684
Obser

In [10]:
discharging_service = services.sort_values(['hadm_id', "transfertime"]).groupby("hadm_id").last().reset_index()

In [11]:
test_dataset_df = test_dataset.to_pandas(); test_dataset_df.shape

(14702, 4)

In [12]:
test_dataset_df = test_dataset_df.merge(discharging_transfer[['hadm_id', 'careunit', 'eventtype']], on="hadm_id", how="left"); test_dataset_df.shape

(14702, 6)

In [13]:
test_dataset_df = test_dataset_df.merge(discharging_service[['hadm_id', 'curr_service']], on="hadm_id", how="left"); test_dataset_df

,note_id,hadm_id,input,output,careunit,eventtype,curr_service
0,19766998-DS-20,26231944,\nName: ___ Unit No: ___\n ...,"___ with PMH HCV, ETOH cirrhosis with ascites,...",Med/Surg,transfer,MED
1,10336082-DS-11,28542384,\nName: ___ Unit No: ___\n \nA...,___ yo F with recent lumbar laminectomy c/b MS...,Neurology,admit,MED
2,10481170-DS-22,26489329,\nName: ___ Unit No: ___\n ...,The patient was seen ___ the emergency departm...,Med/Surg/GYN,transfer,SURG
3,11576109-DS-14,22641254,\nName: ___ Unit No: ___\n ...,"Ms. ___ is a ___ woman with a history of T2DM,...",Medicine/Cardiology,transfer,CMED
4,19249697-DS-17,29265750,\nName: ___ Unit No: ...,___ with IDDM and h/o provoked PE on apixiban ...,Vascular,admit,MED
...,...,...,...,...,...,...,...
14697,18995174-DS-17,27445071,\nName: ___ Unit No: ___\...,___ M PMHx dilated non-ischemic cardiomyopathy...,Medicine/Cardiology,admit,CMED
14698,13588195-DS-9,26192891,\nName: ___ Unit No: ___\n ...,"Mr. ___ is a ___ year-old man with CAD, HTN, H...",Med/Surg,admit,MED
14699,10873131-DS-17,26584893,\nName: ___ Unit No: _...,___ year old female with past medical history ...,Medicine,transfer,MED
14700,12332377-DS-11,25623241,\nName: ___ Unit No: ___\n ...,Mr. ___ is a ___ male with a past medical hist...,Med/Surg/Trauma,admit,MED


In [14]:
test_dataset_df['curr_service'].value_counts()

curr_service
MED      7616
CMED     1340
NMED     1333
SURG     1293
OMED      883
ORTHO     699
NSURG     481
TRAUM     340
VSURG     210
CSURG     132
GYN        99
GU         95
TSURG      87
PSURG      42
OBS        38
PSYCH       8
ENT         6
Name: count, dtype: int64

In [15]:
# print(test_dataset_df[test_dataset_df['careunit'] == "Med/Surg"].sample(1)['output'].squeeze())

test_dataset_df[test_dataset_df['careunit'] == "Hematology/Oncology"]['output'].str.contains("ACUTE ISSUES", case=False).value_counts()

output
False    697
True      17
Name: count, dtype: int64

In [16]:
test_dataset_df

,note_id,hadm_id,input,output,careunit,eventtype,curr_service
0,19766998-DS-20,26231944,\nName: ___ Unit No: ___\n ...,"___ with PMH HCV, ETOH cirrhosis with ascites,...",Med/Surg,transfer,MED
1,10336082-DS-11,28542384,\nName: ___ Unit No: ___\n \nA...,___ yo F with recent lumbar laminectomy c/b MS...,Neurology,admit,MED
2,10481170-DS-22,26489329,\nName: ___ Unit No: ___\n ...,The patient was seen ___ the emergency departm...,Med/Surg/GYN,transfer,SURG
3,11576109-DS-14,22641254,\nName: ___ Unit No: ___\n ...,"Ms. ___ is a ___ woman with a history of T2DM,...",Medicine/Cardiology,transfer,CMED
4,19249697-DS-17,29265750,\nName: ___ Unit No: ...,___ with IDDM and h/o provoked PE on apixiban ...,Vascular,admit,MED
...,...,...,...,...,...,...,...
14697,18995174-DS-17,27445071,\nName: ___ Unit No: ___\...,___ M PMHx dilated non-ischemic cardiomyopathy...,Medicine/Cardiology,admit,CMED
14698,13588195-DS-9,26192891,\nName: ___ Unit No: ___\n ...,"Mr. ___ is a ___ year-old man with CAD, HTN, H...",Med/Surg,admit,MED
14699,10873131-DS-17,26584893,\nName: ___ Unit No: _...,___ year old female with past medical history ...,Medicine,transfer,MED
14700,12332377-DS-11,25623241,\nName: ___ Unit No: ___\n ...,Mr. ___ is a ___ male with a past medical hist...,Med/Surg/Trauma,admit,MED


In [17]:
print(test_dataset_df[test_dataset_df['curr_service'] == "SURG"]['output'].sample(1).squeeze())

The patient was transferred to ___ Emergency Department from 
___ after sustaining a trauma on ___. Non contrast CT scan 
done at ___ was conserning for bladder injury which was not seen 
onrepeat IV cotrast CT at ___. He did have multiple pelvic 
fractures. Cystogram was recommended to rule out bladder injury 
but patient refused foley catheter as well as rectal exam. He 
was admitted to ACS/trauma service for further management and 
treatment. Orthopedic surgery was consulted and recommnedations 
were non operative management of pelvic fracture. touch down 
weight bearing on R. He had a repeat X ray on HD 2, which was 
stable. His UA was negative for hematuria. He was able to 
urinate without problem and post void residuals were normal 
without signs of urinary retention. He was evaluated by physical 
therapy and was able to ambulate with crutches on HD1. His 
tertiary survey did not reveal any new injury.
Neuro: The patient was alert and oriented throughout 
hospitalization; pain wa

In [18]:
test_dataset_df.groupby("curr_service").agg({"output":lambda x: x.str.contains("#").sum()/x.shape[0]}).sort_values('output', ascending=False)

,output
curr_service,
OMED,0.768969
CMED,0.752985
PSYCH,0.750000
MED,0.721376
ORTHO,0.686695
PSURG,0.547619
OBS,0.421053
CSURG,0.363636
NMED,0.363091


In [19]:
test_dataset_df.groupby("careunit").agg({"output":lambda x: x.str.contains("#").sum()}).sort_values('output', ascending=False)

,output
careunit,
Medicine,3099
Med/Surg,1116
Medicine/Cardiology,750
Neurology,562
Hematology/Oncology,555
Med/Surg/Trauma,500
Transplant,457
Med/Surg/GYN,394
Vascular,367


In [20]:
# print out which care units have a # format ordered by which care units are most frequent
test_dataset_df.groupby("careunit").agg({"output":lambda x: x.str.contains("#").sum()/x.shape[0]}).sort_values('output', ascending=False).loc[test_dataset_df.groupby("careunit").agg({"output":lambda x: x.str.contains("#").sum()}).sort_values('output', ascending=False).index]

,output
careunit,
Medicine,0.665450
Med/Surg,0.595836
Medicine/Cardiology,0.746269
Neurology,0.374168
Hematology/Oncology,0.777311
Med/Surg/Trauma,0.465116
Transplant,0.604497
Med/Surg/GYN,0.617555
Vascular,0.558600


In [53]:
print(test_dataset_df[test_dataset_df['careunit'] == "Transplant"]['output'].sample(1).squeeze())

Mr. ___ is a ___ year old male h/o ESRD ___ membranous 
nephropathy s/p graft failure and recent HD re-initiation, 
presenting with hyperkalemia and hematuria, found to have 
rejection of transplant kidney.
 
# Hyperkalemia:  Patient recently admitted with progressive 
renal failure ___ membranous disease s/p tunnelled RIJ placement 
and initiation of HD prior to discharge.  Now re-presenting with 
elevated creatinine and potassium, likely ___ missed dialysis 
session. He received calcium gluconate, insulin+glucose in the 
ED.  He continues on immunosuppression to prevent immune 
reaction from total rejection. Given peaked T waves in the ED, 
he was dialyzed and admitted to the ICU with continued peaked T 
waves despite intervention.  He continued to be asymptomatic in 
this regard.  He was continued on tacrolimus and prednisone as 
above.  Lisinopril was held due to hyperkalemia.  His EKGs 
normalized while in the ICU and he was transferred to the 
Transplant Nephrology floor for furt